In [1]:
import torch

In [3]:
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [13]:
sequence_length = 28
input_size = 28
hidden_size = 128
num_layers = 2
num_classes = 10
batch_size = 100
num_epochs = 2 
learning_rate = 0.003

In [8]:
train_dataset = torchvision.datasets.MNIST(root='../../data/',
                                           train=True, 
                                           transform=transforms.ToTensor(),
                                           download=True)

In [10]:
test_dataset = torchvision.datasets.MNIST(root='../../data/',train=False,transform=transforms.ToTensor())

In [11]:
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,batch_size=batch_size,shuffle=True)

In [12]:
test_loader = torch.utils.data.DataLoader(dataset=test_dataset,batch_size=batch_size,shuffle=False)

In [15]:
class BiRNN(nn.Module):
    def __init__(self,input_size,hidden_size,num_layers,num_classes):
        super(BiRNN,self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size,hidden_size,num_layers,batch_first=True,bidirectional=True)
        self.fc = nn.Linear(hidden_size*2,num_classes)

    def forward(self,x):
        h0 = torch.zeros(self.num_layers*2,x.size(0),self.hidden_size).to(device)
        c0 = torch.zeros(self.num_layers*2,x.size(0),self.hidden_size).to(device)

        out, _ = self.lstm(x,(h0,c0))
        out = self.fc(out[:,-1,:])
        return out

In [16]:
model = BiRNN(input_size,hidden_size,num_layers,num_classes).to(device)

In [17]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr = learning_rate)


In [19]:
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images,labels) in enumerate(train_loader):
        images = images.reshape(-1,sequence_length,input_size).to(device)
        labels = labels.to(device)

        outputs = model(images)
        loss = criterion(outputs,labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if(i+1) %100 == 0:
            print('Epoch [{}/{}],Step [{}/{}],Loss: {:.4f}'.format(epoch+1,num_epochs,i+1,total_step,loss.item()))


Epoch [1/2],Step [100/600],Loss: 0.2045
Epoch [1/2],Step [200/600],Loss: 0.1975
Epoch [1/2],Step [300/600],Loss: 0.1241
Epoch [1/2],Step [400/600],Loss: 0.1420
Epoch [1/2],Step [500/600],Loss: 0.1063
Epoch [1/2],Step [600/600],Loss: 0.0688
Epoch [2/2],Step [100/600],Loss: 0.0455
Epoch [2/2],Step [200/600],Loss: 0.1050
Epoch [2/2],Step [300/600],Loss: 0.0116
Epoch [2/2],Step [400/600],Loss: 0.0941
Epoch [2/2],Step [500/600],Loss: 0.0602
Epoch [2/2],Step [600/600],Loss: 0.0369


In [20]:
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.reshape(-1,sequence_length,input_size).to(device)
        labels = labels.to(device)
        
        outputs = model(images)
        _, predicted = torch.max(outputs.data,1)
        total += labels.size(0)
        correct+= (predicted==labels).sum().item()

    print('Test Accuracy of the model on the 1000 test images: {}'.format(100* correct/total))
    

Test Accuracy of the model on the 1000 test images: 97.5


In [21]:
torch.save(model.state_dict(),'model_BiRNN.ckpt')